In [ ]:
#%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import functions as ff
import data_handling as dh
import importlib
importlib.reload(ff)

In [2]:
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider,FloatSlider
from ipywidgets import BoundedFloatText
from ipywidgets import GridspecLayout, HBox,VBox
from IPython.display import display

In [3]:
t0_box=FloatSlider(description="t0",value=1,min=-2,max=3)
display(t0_box)


FloatSlider(value=1.0, description='t0', max=3.0, min=-2.0)

In [4]:
df=pd.read_csv("results_analyse/cont_r1.csv")
o_obs=dh.unpack_results(df,1,'o',volume=False)
o_x = np.arange(len(o_obs))
o_obs

array([93.0077678 , 93.6976383 , 73.22634605, 58.79950645, 44.74353015,
       35.54030164, 27.9739728 , 24.63892685, 18.23437634, 15.74608407,
       11.20629931,  9.60948579, 13.46979665, 10.35600066])

In [141]:
from ipywidgets import widgets,interactive,interact
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label
from IPython.display import display,clear_output
import matplotlib.pyplot as plt
import numpy as np
global o_obs,o_x

def show_plot():
    plt.close('all')
    plt.ioff()
    fig,ax = plt.subplots(1,1,figsize=(8,6))
    out=widgets.Output()

    res_df = pd.read_pickle("res.pkl")
    def click(b):
        global o_obs,o_x, n_obs,n_x

        Vmax_o = Vmax_o_slide.value
        ao   = ao_slide.value
        to   = to_slide.value
        
        Vmax_n = Vmax_n_slide.value
        a0   = a0_slide.value
        t0   = t0_slide.value
        a1   = a1_slide.value
        t1   = t1_slide.value
        #plt.figure(2)

        
        ax.clear()
        ax.plot(-o_x,o_obs,'o',color='red')
        ax.plot(-n_x,n_obs,'o',color='blue')
        if( True):
            x = np.linspace(-8, 16, num=100)
            ax.plot(-x,ff.rise_only    (x,Vmax_o,ao,-to,1.0),color='red')
            ax.plot(-x,ff.rise_and_fall(x,Vmax_n,a0,-t0,1.0,a1,-t1,1),color='blue')
            ax.plot([to,to],[0,ff.rise_only(-to,Vmax_o,ao,-to,1.0)],ls='--',color='red',lw=1.5)
            
            ax.plot([t0,t0],[0,ff.rise_and_fall(-t0,Vmax_n,a0,-t0,1.0,a1,-t1,1)],ls='--',color='blue',lw=1.5)
            ax.plot([t1,t1],[0,ff.rise_and_fall(-t1,Vmax_n,a0,-t0,1.0,a1,-t1,1)],ls='--',color='blue',lw=1.5)
            
            
            ax.set_xticks(range(-14,2,2))
            ax.set_ylim(0, 200)
            ax.axhline(Vmax_o,ls='--',color='red')
            ax.axvline(0,ls='--',color='grey',lw=1)
            
            [xmin,xmax]=ax.get_xlim()
            [ymin,ymax]=ax.get_ylim()
            ax.annotate(r'$t_o$',(to+0.2,ymax-10))
            ax.annotate(r'$Vmax_{o}}$',(xmax-2,Vmax_o+10))
            


        with out:
            clear_output(wait=True)
            display(ax.figure)

    choice=Dropdown(
        options='cont_r1 cont_r2 cont_r3 caged_d04_r1 caged_d07_r1'.split(),
        value='cont_r1',
        description='Number:',
        disabled=False,
    )
    
    

    
    Vmax_o_slide=FloatSlider(description=r'$V$max$_o$',value=150,min=0,max=300,continuous_update=False)
    Vmax_o_slide.observe(click, names='value')
    ao_slide  =FloatSlider(description=r'$a_o$',value=-0.4,min=-1.5,max=0,continuous_update=False)
    ao_slide.observe(click, names='value')
    to_slide  =FloatSlider(description=r'$t_o$',value=0,min=-4,max=4,continuous_update=False)
    to_slide.observe(click, names='value')
    
    Vmax_n_slide=FloatSlider(description=r'$V$max$_{n}$',value=150,min=0,max=300,continuous_update=False)
    Vmax_n_slide.observe(click, names='value')
    a0_slide  =FloatSlider(description=r'$a_0$',value=-0.4,min=-1.5,max=0,continuous_update=False)
    a0_slide.observe(click, names='value')
    t0_slide  =FloatSlider(description=r'$t_0$',value=-4,min=-8,max=4,continuous_update=False)
    t0_slide.observe(click, names='value')    
    
    a1_slide  =FloatSlider(description=r'$a_1$',value=-0.4,min=-1.5,max=0,continuous_update=False)
    a1_slide.observe(click, names='value')
    t1_slide  =FloatSlider(description=r'$t_1$',value=0,min=-4,max=4,continuous_update=False)
    t1_slide.observe(click, names='value')       
    
    
    def choice_selected(b):
        global o_obs,o_x, n_obs,n_x
        name = choice.value
        df=pd.read_csv("results_analyse/{}.csv".format(name))
        o_obs=dh.unpack_results(df,1,'o',volume=False)
        o_x = np.arange(len(o_obs))
        n_obs=dh.unpack_results(df,1,'n',volume=False)
        n_x = np.arange(len(n_obs))
        
        rown = "{}_1".format(name)
        Vmax_n_slide.value= res_df.loc[rown,('Vmax_n','m')]
        a0_slide.value= res_df.loc[rown,('a0','m')]
        t0_slide.value= -res_df.loc[rown,('t0','m')]
        a1_slide.value= res_df.loc[rown,('a1','m')]
        t1_slide.value= -res_df.loc[rown,('t1','m')]
        
        Vmax_o_slide.value= res_df.loc[rown,('Vmax_o','m')]
        ao_slide.value= res_df.loc[rown,('a_o','m')]
        to_slide.value= -res_df.loc[rown,('t_o','m')]
        click(None)
        #f(Vmax_slide.value, a0_slide.value, t0_slide.value)
        return
    choice_selected(None)
    choice.observe(choice_selected)
        
    
    
    #display(VBox([mslide,cslide]))
    oocyte_params=widgets.VBox([Label(value="Oocyte"),Vmax_o_slide,ao_slide,to_slide])
    nurse_params =widgets.VBox([Vmax_n_slide,a0_slide,t0_slide,a1_slide,t1_slide])
    box = widgets.VBox([choice,widgets.HBox([oocyte_params,nurse_params]),out])
    display(box)
    
    
    click(None)
    #interact(f,Vmax=Vmax_slide,a0=a0_slide,t0=t0_slide);


In [142]:
show_plot()

In [134]:
res_df

Vmax_o                         a_o                      \
                      m       lo       hi         m        lo        hi   
cont_r1_1       113.708  86.7459  171.602 -0.496073  -0.72075 -0.282157   
cont_r2_1       168.705  121.913  260.121 -0.638321  -0.97021 -0.392384   
cont_r2_2       158.233  112.862  274.149 -0.777375  -1.22541 -0.407417   
cont_r3_1       134.244  94.3304  215.622 -0.526833 -0.758632 -0.334343   
cont_r3_2       125.533  87.2236  208.014 -0.532076 -0.875509 -0.305781   
cont_r3_3       126.901  98.3261  179.906 -0.548357 -0.801012 -0.340446   
caged_d10_r1_3   70.523  53.2381  153.411  -1.77104  -3.04591 -0.554981   
caged_d10_r1_1  179.569  70.8633  449.734  -1.12042  -1.84546 -0.641523   
caged_d10_r1_2  210.702  96.8471  501.805   -1.7796  -2.83235 -0.967197   
caged_d10_r1_4  83.5496  55.1998  155.212  -1.10957  -2.00614 -0.518787   
caged_d10_r3_1  175.269  73.1846  387.253  -1.62015   -2.9739 -0.847503   
caged_d10_r3_2  104.976   46.056  259.717 -0.546464  -1.01322 -0.293773   
caged_d10_r3_3  158.866  71.6642  383.729  -1.86364  -3.16569  -0.89805   
caged_d04_r1_1  289.481  126.617  571.937 -0.590043 -0.878689  -0.38097   
caged_d04_r1_2  110.995  74.2477  185.705 -0.674913  -1.13393 -0.366251   
caged_d04_r2_1  126.198  62.7482  257.061 -0.632815 -0.971887 -0.361129   
caged_d04_r2_2  176.161  62.7087  374.862 -0.506272 -0.802907 -0.274373   
caged_d04_r2_3  187.653   58.896  377.434  -0.60836 -0.935851 -0.365719   
caged_d04_r2_4  72.0376  52.7321  128.873 -0.792575  -1.36577 -0.273092   
caged_d04_r2_5  106.639  60.7501  233.279 -0.637409  -1.32106 -0.263228   
caged_d07_r1_1  74.8615  55.4255  135.866  -1.10408  -2.29623 -0.362348   
caged_d07_r1_2  98.9326  52.9762  216.015 -0.697803  -1.29926 -0.376652   
caged_d07_r1_3  181.618  67.4889  433.023 -0.828414  -1.38976 -0.442321   

                      t_o                         nu_o  ...           a1  \
                        m        lo        hi        m  ...           hi   
cont_r1_1         2.58959  0.407097   3.56701  1.09752  ...    -0.387501   
cont_r2_1         1.37942 -0.336786   2.30887  1.16655  ...      -2.2876   
cont_r2_2         1.46866 -0.351767   2.47813  1.13571  ...     -1.26368   
cont_r3_1         1.38649 -0.860039   2.65352  1.17964  ...     -1.98146   
cont_r3_2         2.29106 -0.124318   3.67943  1.08455  ...     -1.98728   
cont_r3_3         2.00541  0.456872     2.967  1.19243  ...     -2.38118   
caged_d10_r1_3    1.46347  -0.24858   1.96743  1.07214  ...     -3.29954   
caged_d10_r1_1  -0.339378  -1.91327   1.04612  1.12265  ...     -1.88895   
caged_d10_r1_2  -0.140218  -1.14679  0.623822  1.11821  ... -3.24047e-05   
caged_d10_r1_4    1.27924 -0.391635   2.24015  1.01955  ...  -0.00805361   
caged_d10_r3_1  -0.131329  -1.22453  0.838155  1.06272  ...  -0.00013785   
caged_d10_r3_2   0.326936   -3.1887   3.00142  1.03116  ...    -0.203874   
caged_d10_r3_3 -0.0253489  -1.03406  0.851448  1.07033  ... -3.92522e-06   
caged_d04_r1_1   -1.57372  -3.93557  0.371449  1.25237  ...     -3.41703   
caged_d04_r1_2     1.6747 -0.361354   2.88293  1.12151  ...     -2.63146   
caged_d04_r2_1  0.0806071  -2.47451   1.72485  1.11109  ...    -0.227798   
caged_d04_r2_2   -1.07349  -3.97851   1.78743  1.15088  ...      -1.2638   
caged_d04_r2_3   -1.37422  -3.59541   1.27414  1.16461  ... -4.61749e-05   
caged_d04_r2_4    2.50154  0.175506   3.33995    1.093  ...     -1.92483   
caged_d04_r2_5    1.13896  -1.97933   2.81983  1.07412  ...    -0.829792   
caged_d07_r1_1    2.09238  0.259196   3.13324   1.0678  ...     -2.48892   
caged_d07_r1_2   0.660759  -2.00614   2.15767  1.08953  ...     -2.93784   
caged_d07_r1_3  -0.631094  -2.71815   1.33245  1.13867  ... -0.000124685   

                       t1                              nu1                     \
                        m           lo        hi         m        lo       hi   
cont_r1_1         1.34891     0.457062   2.38211    1.0027  0.577709  1.55834 